In [1]:
# import libraries
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime

import tensorflow as tf
import pandas as pd
import numpy as np

# for NLP
# from konlpy.tag import Kkma
# from konlpy.utils import pprint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno

plt.style.use('seaborn')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
font_name = matplotlib.font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
matplotlib.rc('font', family = font_name)

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.DataFrame(song_meta)
train = pd.DataFrame(train)
valid = pd.DataFrame(valid)

# 1. tag dataframe 제작

In [3]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [4]:
plylst_tag_map

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


In [5]:
# 태그 별 매핑 빈도 수 저장 
tag_cnt = plylst_tag_map.groupby('tags').tags.count().reset_index(name = 'mapping_cnt')
tag_cnt['tags'] = tag_cnt['tags'].astype(str)
tag_cnt['mapping_cnt'] = tag_cnt['mapping_cnt'].astype(int)

In [6]:
tag_cnt

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
...,...,...
29155,힙해,5
29156,힙힙힙,2
29157,힛뎀포크,1
29158,힛뎀폭,1


In [7]:
tag_cnt['tags'][0]
#,, 이런식으로 아무것도 안들어있는건가..?

''

In [9]:
#일단 만들자
#dataframe으로 저장
tag_cnt.to_json('./res/tag_cnt.json', orient='table')

In [11]:
test_print=pd.read_json('./res/tag_cnt.json', orient='table')
test_print

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
...,...,...
29155,힙해,5
29156,힙힙힙,2
29157,힛뎀포크,1
29158,힛뎀폭,1
